In [1]:
import time, cv2, sys, os, torch
from threading import Thread
from djitellopy import Tello
import openai
import re
import torch
import time

# url to CV2 image
import urllib.request
import numpy as np
import cv2


# load environment variables
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/prompts/")

from vqa_question.v1 import prompt_template as vqa_prompt_template
from drone_explore_prompt import prompt_template as explore_template
from drone_next_action.v1 import prompt_template as drone_prompt_template

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/Scripts/")

from data_collection_utils import take_individual_picture, stream_video, stream_frames, record_streamed_frames

print("Imported all modules")


Imported all modules


In [2]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
# pred = model.predict('https://ultralytics.com/images/zidane.jpg')


In [3]:
def get_coords(obj_name, img_url, verbose=True): # obj_name must be a valid YOLO class
    start_time = time.time()
    pred = model(img_url)
    
    if verbose:
        print(pred)
        # pred.show()

    # get bouning box coordinates for object
    classes = pred[0].boxes.cls

    idx_to_name = pred[0].names

    names = [idx_to_name[int(i)] for i in classes]
    if obj_name not in names:
        if verbose:
            print(names)
        print(time.time() - start_time, 'seconds elapsed')
        
        return None, None
    idx_obj = names.index(obj_name)
    coords = pred[0].boxes.xyxy[idx_obj]
    if verbose:
        print(names)
    print(time.time() - start_time, 'seconds elapsed')
    return (int(coords[0].item()), int(coords[1].item())), (int(coords[2].item()), int(coords[3].item()))
    

    # classes = pred[0].boxes.cls
    
    # idx_to_name = pred[0].names

    # names = [idx_to_name[int(i)] for i in classes]
    # if obj_name not in names:
    #     if verbose:
    #         print(names)
    #     print(time.time() - start_time, 'seconds elapsed')
    #     return None
    # idx_obj = names.index(obj_name)
    # coords = pred[0].boxes.xyxy[idx_obj]
    # if verbose:
    #     print(names)
    # print(time.time() - start_time, 'seconds elapsed')
    # return (coords[0], coords[1]), (coords[2], coords[3])

In [4]:
# top_left, bottom_right = get_coords('person', 'https://ultralytics.com/images/zidane.jpg')

In [5]:
# import logging 
# Tello.LOGGER.setLevel(logging.DEBUG)

In [6]:
def plot_box(img, top_left, bottom_right):
    # req = urllib.request.urlopen(img_url)
    # arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    # img = cv2.imdecode(arr, -1) # 'Load it as it is'
    # cv2.imshow("something", img)
    rect = cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)
    cv2.imshow("something", rect)
    cv2.waitKey(1)

In [7]:
# plot_box("https://ultralytics.com/images/zidane.jpg", top_left=top_left, bottom_right=bottom_right)

In [8]:
drone = Tello()
drone.connect()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


In [9]:
# # Mission: Dr0n3 sees object and if it is what it was looking for, it will fly to it
# drone.streamon()
# time.sleep(5)
# img = drone.get_frame_read().frame
# # img = drone.get_frame_read().frame
# img = cv2.resize(img, (360, 240))
# # cv2.imshow("image", img)
# # cv2.waitKey(1)

# top_left, bottom_right = get_coords("bottle", img)
# cv2.imshow("image", img)
# cv2.waitKey(1)
# if top_left is not None:
#     print("something found")
#     plot_box(img, top_left=top_left, bottom_right=bottom_right)
drone.streamon()
time.sleep(5)
while True:
    img = drone.get_frame_read().frame
    img = cv2.resize(img, (360, 240))
    top_left, bottom_right = get_coords("bottle", img)
    cv2.imshow("image", img)
    cv2.waitKey(1)
    if top_left is not None:
        print("Bottle Found!")
        plot_box(img, top_left=top_left, bottom_right=bottom_right)
    time.sleep(2)
    
    


[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'
[h264 @ 0x7fa372e960c0] error while decoding MB 8 3, bytestream -5
[h264 @ 0x7fa372e8b940] error while decoding MB 9 24, bytestream -6
[h264 @ 0x7fa372e6c280] left block unavailable for requested intra mode
[h264 @ 0x7fa372e6c280] error while decoding MB 0 26, bytestream 561
Ultralytics YOLOv8.0.40 🚀 Python-3.9.13 torch-1.13.1 CPU
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 448x640 2 persons, 1 bottle, 2 chairs, 276.3ms
Speed: 2.0ms pre-process, 276.3ms inference, 8.7ms postprocess per image at shape (1, 3, 640, 640)


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([5, 6])
dtype: torch.float32
 + tensor([[1.83000e+02, 1.40000e+01, 2.16000e+02, 1.03000e+02, 8.18590e-01, 3.90000e+01],
        [1.60000e+02, 0.00000e+00, 3.59000e+02, 1.66000e+02, 6.47017e-01, 0.00000e+00],
        [6.40000e+01, 5.00000e+00, 3.59000e+02, 1.39000e+02, 4.28061e-01, 0.00000e+00],
        [0.00000e+00, 0.00000e+00, 9.20000e+01, 1.35000e+02, 3.31152e-01, 5.60000e+01],
        [8.60000e+01, 0.00000e+00, 1.47000e+02, 5.60000e+01, 3.02267e-01, 5.60000e+01]])]
['bottle', 'person', 'person', 'chair', 'chair']
0.7374520301818848 seconds elapsed
something found


[h264 @ 0x7fa372dcae00] error while decoding MB 0 38, bytestream -6
[h264 @ 0x7fa372dcae00] error while decoding MB 55 18, bytestream -6
[h264 @ 0x7fa372e76a00] error while decoding MB 22 44, bytestream -8
[h264 @ 0x7fa372dcae00] error while decoding MB 27 20, bytestream -6
[h264 @ 0x7fa372e8b940] error while decoding MB 28 35, bytestream -5
[h264 @ 0x7fa372e960c0] error while decoding MB 45 36, bytestream -6
[h264 @ 0x7fa372de5240] error while decoding MB 22 32, bytestream -8
[h264 @ 0x7fa372e81180] left block unavailable for requested intra mode
[h264 @ 0x7fa372e81180] error while decoding MB 0 34, bytestream 6478
[h264 @ 0x7fa372e960c0] error while decoding MB 47 29, bytestream -6
[h264 @ 0x7fa372dd0340] left block unavailable for requested intra mode
[h264 @ 0x7fa372dd0340] error while decoding MB 0 42, bytestream 817
[h264 @ 0x7fa372e81180] error while decoding MB 9 35, bytestream -6
[h264 @ 0x7fa372e81180] left block unavailable for requested intra mode
[h264 @ 0x7fa372e81180] er

In [13]:
cv2.imshow('a', img)
cv2.waitKey(1)

-1

[h264 @ 0x7f8d9b7075c0] error while decoding MB 37 19, bytestream -6
[h264 @ 0x7f8dbbcdd900] error while decoding MB 22 39, bytestream -10
[h264 @ 0x7f8dbbcf2840] error while decoding MB 55 38, bytestream -6
[h264 @ 0x7f8dbdfee640] error while decoding MB 16 33, bytestream -6
[h264 @ 0x7f8dbdff8dc0] error while decoding MB 19 34, bytestream -6
[h264 @ 0x7f8dbbcf2840] error while decoding MB 2 39, bytestream -6
[h264 @ 0x7f8dbbcdd900] error while decoding MB 2 44, bytestream -6
[h264 @ 0x7f8dbdfee640] error while decoding MB 48 34, bytestream -6
[h264 @ 0x7f8dbbcf2840] left block unavailable for requested intra mode
[h264 @ 0x7f8dbbcf2840] error while decoding MB 0 31, bytestream 1474
[h264 @ 0x7f8dbbcd3500] error while decoding MB 56 25, bytestream -6
[h264 @ 0x7f8d9b7075c0] left block unavailable for requested intra mode
[h264 @ 0x7f8d9b7075c0] error while decoding MB 0 29, bytestream 1491
[h264 @ 0x7f8d9b71c4c0] error while decoding MB 16 23, bytestream -8
[h264 @ 0x7f8dbbcf2840] err

In [10]:
# !pip freeze > requirements.txt

In [86]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.


In [5]:
drone.connect()

[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds


Exception: Command 'command' was unsuccessful for 4 tries. Latest response:	'Aborting command 'command'. Did not receive a response after 7 seconds'

In [6]:
take_individual_picture(drone)

[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


In [ ]:
stream = Thread(target=stream_frames, args=(drone, False))
stream.start()

[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'error'


[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'error'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'error'
Exception in thread Thread-8 (stream_frames):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sidbendre/project/DroneFormer/Scripts/data_collection_utils.py", line 19, in stream_frames
    drone.streamon()
  File "/opt/homebrew/lib/python3.10/site-packages/djitellopy/enforce_types.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/opt/homebrew/lib/python3.10/site-packages/djitellopy/tello.py", line 593, in streamon
    self.s

: 

In [4]:
objective = "Find the person who is wearing a blue cap"

In [5]:
drone.streamon()
drone.takeoff()

[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'streamon'
[WARNING] tello.py - 447 - Aborting command 'streamon'. Did not receive a response after 7 seconds


Exception: Command 'streamon' was unsuccessful for 4 tries. Latest response:	'Aborting command 'streamon'. Did not receive a response after 7 seconds'

In [32]:
def query_the_environment(drone, question):
    image = drone.get_frame_read().frame
    encoding = processor(image, question, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = torch.sigmoid(logits).argmax(-1).item()
    return str(model.config.id2label[idx])


In [33]:
# Generate next action prompt
def next_action_prompt(prompt_template, objective, environment_context, previous_commands, previous_context, vqa_questions):
    prompt = prompt_template.replace("$objective", objective)
    prompt = prompt.replace("$context", environment_context)
    prompt = prompt.replace("$previous_commands", previous_commands)
    prompt = prompt.replace("$previous_context", previous_context)
    prompt = prompt.replace("$vqa_questions", vqa_questions)
    return prompt, ["\n"] # prompt and stop token

In [34]:
def prompt(prompt, stop_tokens):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.5,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=10,
        stop=stop_tokens,
    )
    return response.choices[0].text

In [35]:
def get_vqa_questions(vqa_prompt_template, objective):
    prompt_text = vqa_prompt_template.replace("$objective", objective)
    result = prompt(prompt_text, ["&&&&&&"])
    matches = re.findall(r'@(.+?)@', result)
    return matches


In [36]:
get_vqa_questions(vqa_prompt_template, objective)

['Is there a person in the image?', 'Is there a blue cap in the image?']

In [ ]:
def explore(drone, use_gpt=False, prompt_template=explore_template):
    if use_gpt:
       prompt(prompt_template)
    else:
        import random as r
        # randomly generate a number between 45 and 180
        random_angle = r.randint(45, 180)
        # randomly generate a number between 5 and 20
        random_distance = r.randint(5, 20)
        actions = [(drone.move_left, random_distance), (drone.move_right, random_distance), (drone.rotate_clockwise, random_angle), (drone.rotate_counter_clockwise, random_angle), (drone.move_forward, random_distance), (drone.move_back, random_distance)]

        # sample 3 actions
        sampled_actions = r.sample(actions, 3)
        for action in sampled_actions:
            action[0](action[1])
            time.sleep(0.5)


In [1]:
# Get the 4 points of a bounding box
def find_corners(upper_left, length: float, width: float):
    corners = []
    
    upper_right = (upper_left[0] + length, upper_left[1])
    lower_left =  (upper_left[0], upper_left[1] + width)
    lower_right = (upper_left[0] + length, upper_left[1] + width)
    
    corners.append(upper_left)
    corners.append(upper_right)
    corners.append(lower_left)
    corners.append(lower_right)
    
    return corners
    
# Find the center of a bounding box
def find_center(corners):
    upper_left, upper_right, lower_left, lower_right = corners
    
    # create diagonal using upper left and lower right
    center_x = (upper_left[0] + upper_right[0]) / 2
    center_y = (upper_left[1] + lower_right[1]) / 2
    
    return(center_x, center_y)


# Exploit Function: Input we have a bounding box with 4 corners (or 1 corner + length and width for a more compact representation)
def exploit(drone, bbox, use_gpt = False, prompt_template = explore_template):
    if use_gpt:
        prompt(prompt_template)
        
    distance_to_object = None
    angle_to_object = None
    height_to_object = None
    actions = [
        (drone.move_left, distance_to_object),
        (drone.move_right, distance_to_object),
        (drone.rotate_clockwise, angle_to_object),
        (drone.rotate_counter_clockwise, angle_to_object),
        (drone.move_forward, distance_to_object),
        (drone.move_back, distance_to_object) 
    ]
        
    # Step 1: Find out whether or not the drone is facing the object
    object_in_frame = False
    while object_in_frame == False:
        drone.rotate_clockwise(30)
    
    
    # Step 2: Translate the object to the center
    bounding_box_center = find_center(bbox)
    current_image = drone.get_frame_read().frame
    height, width, channel = current_image.shape
    frame_center = (width / 2, height / 2)
    
    delta_y = frame_center[1] - bounding_box_center[1]
    delta_x = frame_center[0] - bounding_box_center[0]
    
    if delta_y < 0:
        drone.move_up(abs(delta_y))
    else:
        drone.move_down(abs(delta_y))
    
    if delta_x < 0:
        drone.move_left(abs(delta_x))
    else:
        drone.move_right(abs(delta_x))
    
    # Step 3: Zoom In (move towards the object)
    bbox_length = bbox[1][0] - bbox[0][0]
    bbox_width = bbox[1][1] - bbox[0][1]
    
    while bbox_width <= 100 or bbox_length <= 100:
        drone.move_forward(10)
    
    # Step 4: Terminate
    return

NameError: name 'explore_template' is not defined